Creating JSON

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!cp -R '/content/drive/MyDrive/Colab Notebooks/Project/noise-reducer-ml/Dataset' /content/

In [ ]:
%cd /content/Dataset

/content/Dataset


In [ ]:
import json
import logging
import os
import re


def make_json_from_dir(dir, json_dir, file_format = {'.wav'}):
	file_list = os.listdir(dir)
	path_list = []
	for file_name in file_list:
		if os.path.splitext(file_name)[-1] in file_format:
			path_list += [os.path.join(dir, file_name)]
	#print(path_list)
	with open(json_dir, 'w') as f:
		json.dump(path_list, f, indent=4)

In [ ]:
make_json_from_dir('/content/Dataset/noisy', '/content/Dataset/noisy.json')
make_json_from_dir('/content/Dataset/clean', '/content/Dataset/clean.json')

In [ ]:
import json
import os
dir = '/content/Dataset/noisy'
file_list = os.listdir(dir)

In [ ]:
path_list = []
for file_name in file_list:
  if file_name[-3:] == 'wav':
    path_list += [os.path.join(dir, file_name)]
#print(path_list)
with open('/content/Dataset/noisy.json', 'w') as f:
    json.dump(path_list, f, indent=4)

In [ ]:
import json
import os
dir = '/content/Dataset/clean'
file_list = os.listdir(dir)

In [ ]:
path_list = []
for file_name in file_list:
  if file_name[-3:] == 'wav':
    path_list += [os.path.join(dir, file_name)]
#print(path_list)
with open('/content/Dataset/clean.json', 'w') as f:
    json.dump(path_list, f, indent=4)

In [ ]:
import json
import logging
import os
import re

def match_dns(noisy, clean):
	"""match_dns.
	Match noisy and clean DNS dataset filenames.
	:param noisy: list of the noisy filenames
	:param clean: list of the clean filenames
	"""
	print("Matching noisy and clean for dns dataset")
	noisydict = {}
	extra_noisy = []
	for path in noisy:
		print(path)
		match = re.search(r'fileid_(\d+)\.wav$', path)
		if match is None:
			# maybe we are mixing some other dataset in
			extra_noisy.append(path)
		else:
			noisydict[match.group(1)] = path
	noisy[:] = []
	extra_clean = []
	copied = list(clean)
	clean[:] = []
	for path in copied:
		print(path)
		match = re.search(r'fileid_(\d+)\.wav$', path)
		if match is None:
			extra_clean.append(path)
		else:
			noisy.append(noisydict[match.group(1)])
			clean.append(path)
	extra_noisy.sort()
	extra_clean.sort()
	clean += extra_clean
	noisy += extra_noisy

def modify(noisy_locaton, clean_location):
	with open(noisy_locaton, 'r+') as ns , open(clean_location, 'r+') as cl:
		data_noisy = json.load(ns)
		data_clean = json.load(cl)
		match_dns(data_noisy, data_clean)
		ns.seek(0) # <--- should reset file position to the beginning.
		cl.seek(0) # <--- should reset file position to the beginning.
		json.dump(data_noisy, ns, indent=4)
		json.dump(data_clean, cl, indent=4)
		ns.truncate() # remove remaining part
		cl.truncate() # remove remaining part


In [ ]:
modify('/content/Dataset/noisy.json','/content/Dataset/clean.json')

Matching noisy and clean for dns dataset
/content/Dataset/noisy/copy1_p287_005.wav
/content/Dataset/noisy/copy1_p287_001.wav
/content/Dataset/noisy/copy1_p287_006.wav
/content/Dataset/noisy/p287_006.wav
/content/Dataset/noisy/p287_002.wav
/content/Dataset/noisy/p287_005.wav
/content/Dataset/noisy/p287_003.wav
/content/Dataset/noisy/copy1_p287_002.wav
/content/Dataset/noisy/copy1_p287_003.wav
/content/Dataset/noisy/p287_001.wav
/content/Dataset/noisy/copy1_p287_004.wav
/content/Dataset/noisy/p287_004.wav
/content/Dataset/clean/copy1_p287_005.wav
/content/Dataset/clean/copy1_p287_001.wav
/content/Dataset/clean/copy1_p287_006.wav
/content/Dataset/clean/p287_006.wav
/content/Dataset/clean/p287_002.wav
/content/Dataset/clean/p287_005.wav
/content/Dataset/clean/p287_003.wav
/content/Dataset/clean/copy1_p287_002.wav
/content/Dataset/clean/copy1_p287_003.wav
/content/Dataset/clean/p287_001.wav
/content/Dataset/clean/copy1_p287_004.wav
/content/Dataset/clean/p287_004.wav


Converting data to TFrecord file

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/Project/noise-reducer-ml/denoiser-inverse/

/content/drive/MyDrive/Colab Notebooks/Project/noise-reducer-ml/denoiser-inverse


In [2]:
%cd /content/sample_data

/content/sample_data


In [10]:
from tqdm import tqdm
!python3 test.py -n '/content/Dataset/noisy.json' -c '/content/Dataset/clean.json' -o '/content/Dataset/tfrecords' --num-shards-train 3 --num-shards-test 1 --num-shards-val 1 --duration 4 --sample-rate 48000 --test-size 0.2 --val-size 0.2




 Namespace(clean_json='/content/Dataset/clean.json', duration=4, n_shards_test=1, n_shards_train=3, n_shards_val=1, noisy_json='/content/Dataset/noisy.json', output_dir='/content/Dataset/tfrecords', sample_rate=48000, test_size=0.2, val_size=0.2) 



Converting train set into TFRecord shards...
  0% 0/3 [00:00<?, ?it/s]
  0% 0/2 [00:00<?, ?it/s]2022-04-11 05:38:05.935986: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected

 50% 1/2 [00:00<00:00,  4.22it/s]
100% 2/2 [00:00<00:00,  3.51it/s]
 33% 1/3 [00:00<00:01,  1.79it/s]
  0% 0/2 [00:00<?, ?it/s]
 50% 1/2 [00:00<00:00,  2.72it/s]
100% 2/2 [00:00<00:00,  2.63it/s]
 67% 2/3 [00:01<00:00,  1.48it/s]
  0% 0/2 [00:00<?, ?it/s]
 50% 1/2 [00:00<00:00,  2.74it/s]
100% 2/2 [00:00<00:00,  2.67it/s]
100% 3/3 [00:02<00:00,  1.45it/s]
Converting test set into TFRecord shards...
  0% 0/1 [00:00<?, ?it/s]
  0% 0/3 [00:00<?, ?it/s]
 33% 1/3 [00:00<00:00,  4.78it/s]

Copy Back to Drive

In [ ]:
!cp -R /content/Dataset '/content/drive/MyDrive/Colab Notebooks/Project/noise-reducer-ml'

In [ ]:
%cd /content/Dataset

/content/Dataset
